In [ ]:
### EXTRAI AS HARMONIZAÇÕES LISTADAS NO SITE BREJAS PARA UM CSV ###

In [89]:
# imports necessários

import pandas as pd
import requests
from bs4 import BeautifulSoup
from translatepy import Translator
import numpy as np

In [52]:
# processo de web scrapping da pagina que contem lista de harmonizacoes

# URL da página a ser raspada
url = "https://www.brejas.com.br/harmonizacao/harmonizacao-cerveja-pratos.shtml#u"

# Fazer o request para a página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Encontrar a tabela com as preparações e estilos de cerveja
tabela = soup.find("table")

# Criar listas para armazenar os dados
preparacoes = []
estilos_cerveja = []
categoria = []

# Percorrer as linhas da tabela
for linha in tabela.find_all("tr")[1:]:  # Ignorar o cabeçalho
    colunas = linha.find_all("td")
    if len(colunas) > 1:  # Verificar se há dados suficientes
        preparacao = colunas[0].text.strip()
        estilos = colunas[1].text.strip().replace('\n', ', ')  # Substituir '\n' por vírgulas
        preparacoes.append(preparacao)
        estilos_cerveja.append(estilos)

# Criar um DataFrame com os dados
df = pd.DataFrame({
    "pratos": preparacoes,
    "estilos_cervejas": estilos_cerveja
})

df = df.dropna(subset=['estilos_cervejas'])
df = df.dropna(subset=['pratos'])

# Mostrar as primeiras linhas do DataFrame
print(df.head())

# Salvar o DataFrame em um arquivo CSV
df.to_csv("./datasets/raw/HARMONIZACOES.csv", index=False)


                          pratos  \
0  Abacate (Guacamole ou Salada)   
1              Amêndoas Salgadas   
2                       Anchovas   
3                  Appfelstrudel   
4      Arenque (Peixe gorduroso)   

                                    estilos_cervejas  
0  American Pale Ale, India Pale Ale (IPA), Amber...  
1              American Pale Ale, American Brown Ale  
2                   German Pilsner, Bohemian Pilsner  
3       Russian Imperial Stout , Sweet Stout, Porter  
4  Flanders Red Ale , Flanders Brown,            ...  


In [12]:
### PRE TRATAMENTO DOS DADOS ###

In [ ]:
# INVERTENDO ORDEM DAS COLUNAS PARA LISTAR AS CERVEJAS COM SEUS RESPECTIVOS PRATOS 

# Criar um dicionário para armazenar as preparações por estilo de cerveja
estilos_dict = {}

# Iterar pelas linhas do DataFrame
for index, row in df.iterrows():
    preparacao = row['pratos']
    estilos = row['estilos_cervejas'].split(', ')  # Separar os estilos por vírgula
    categoria = row['estilos_cervejas'].split(', ')  # Separar os estilos por vírgula

    # Adicionar as preparações a cada estilo no dicionário
    for estilo in estilos:
        estilo = estilo.strip()  # Remover espaços em branco
        if estilo not in estilos_dict:
            estilos_dict[estilo] = set()  # Criar um conjunto vazio se o estilo não existir
        estilos_dict[estilo].add(preparacao)  # Adicionar a preparação ao conjunto do estilo

# Criar um novo DataFrame a partir do dicionário
novos_dados = {
    "estilos_cervejas": [],
    "pratos": []
}

# Preencher o novo DataFrame
for estilo, preparacoes in estilos_dict.items():
    novos_dados["estilos_cervejas"].append(estilo)
    novos_dados["pratos"].append(preparacoes)  # Adicionar o conjunto de preparações

# Converter os conjuntos para strings para que possam ser salvos em CSV
harmonizacao_df = pd.DataFrame(novos_dados)
harmonizacao_df['pratos'] = harmonizacao_df['pratos'].apply(lambda x: str(x))  # Converte o conjunto em string

# Exibir as primeiras linhas do novo DataFrame
print(harmonizacao_df.head())


In [54]:
# INVERTENDO ORDEM DAS COLUNAS PARA LISTAR AS CERVEJAS COM SEUS RESPECTIVOS PRATOS 

# Criar um dicionário para armazenar as preparações por estilo de cerveja
estilos_dict = {}

# Iterar pelas linhas do DataFrame
for index, row in df.iterrows():
    preparacao = row['pratos']
    estilos = row['estilos_cervejas'].split(', ')  # Separar os estilos por vírgula

    # Adicionar as preparações a cada estilo no dicionário
    for estilo in estilos:
        estilo = estilo.strip()  # Remover espaços em branco
        if estilo not in estilos_dict:
            estilos_dict[estilo] = set()  # Criar um conjunto vazio se o estilo não existir
        estilos_dict[estilo].add(preparacao)  # Adicionar a preparação ao conjunto do estilo

# Criar um novo DataFrame a partir do dicionário
novos_dados = {
    "estilos_cervejas": [],
    "pratos": []
}

# Preencher o novo DataFrame
for estilo, preparacoes in estilos_dict.items():
    novos_dados["estilos_cervejas"].append(estilo)
    novos_dados["pratos"].append(preparacoes)  # Adicionar o conjunto de preparações

# Converter os conjuntos para strings para que possam ser salvos em CSV
harmonizacao_df = pd.DataFrame(novos_dados)
harmonizacao_df['pratos'] = harmonizacao_df['pratos'].apply(lambda x: str(x))  # Converte o conjunto em string

# Exibir as primeiras linhas do novo DataFrame
print(harmonizacao_df.head())


       estilos_cervejas                                             pratos
0     American Pale Ale  {'Atum (massas ou sanduíches)', 'Enchiladas (T...
1  India Pale Ale (IPA)  {'Chouriço', 'Enchiladas (Tortilhas)', 'Carne ...
2           Amber Lager  {'Lasanha', 'Abacate (Guacamole ou Salada)', '...
3    American Brown Ale  {'Feijão/ Feijoada', 'Carne com Chili', 'Bife ...
4        German Pilsner  {'Sopa (caldo de Piranha)', 'Caranguejo', 'Emp...


In [56]:
# CRIA COLUNA DE PRATOS TRADUZIDOS PARA O INGLES

# Inicializa o tradutor
translator = Translator()

# Função para traduzir uma lista de preparações
def translate_preparations(preparations):
    # Verifica se preparations é um conjunto em formato de string e converte para lista
    if isinstance(preparations, str):
        preparations = eval(preparations)  # Converte a string do conjunto de volta para um conjunto

    # Traduz cada preparação individualmente
    translations = {translator.translate(preparation, destination_language='en').result for preparation in preparations}
    return translations  # Retorna um conjunto

# Adiciona uma nova coluna 'translated_dishes' com as traduções
harmonizacao_df['pratos_en'] = harmonizacao_df['pratos'].apply(translate_preparations)

# Exibe o DataFrame resultante
print(harmonizacao_df)


                  estilos_cervejas  \
0                American Pale Ale   
1             India Pale Ale (IPA)   
2                      Amber Lager   
3               American Brown Ale   
4                   German Pilsner   
..                             ...   
63                                   
64                     Traditional   
65                            Bock   
66             Belgian Dark Strong   
67  Fruit Beer(Framboise ou Kriek)   

                                               pratos  \
0   {'Atum (massas ou sanduíches)', 'Enchiladas (T...   
1   {'Chouriço', 'Enchiladas (Tortilhas)', 'Carne ...   
2   {'Lasanha', 'Abacate (Guacamole ou Salada)', '...   
3   {'Feijão/ Feijoada', 'Carne com Chili', 'Bife ...   
4   {'Sopa (caldo de Piranha)', 'Caranguejo', 'Emp...   
..                                                ...   
63                               {'Salame', 'Vitela'}   
64                               {'Tamboril (peixe)'}   
65                            

In [58]:
harmonizacao_df.rename(columns={'pratos': 'pratos_pt'}, inplace=True)


In [60]:
harmonizacao_df.rename(columns={'pratos_en': 'pratos'}, inplace=True)


In [62]:
# renomear todas as linhas que contem 'IPA' no nome para apenas 'IPA'
harmonizacao_df.loc[harmonizacao_df['estilos_cervejas'].str.contains('IPA', case=False, na=False), 'estilos_cervejas'] = 'IPA'


In [64]:
harmonizacao_df.head()

,estilos_cervejas,pratos_pt,pratos
0,American Pale Ale,"{'Atum (massas ou sanduíches)', 'Enchiladas (T...","{Flooded (tortillas), Empanadas, Tuna (pasta o..."
1,IPA,"{'Chouriço', 'Enchiladas (Tortilhas)', 'Carne ...","{Avocado (guacamole or salad), Flooded (tortil..."
2,Amber Lager,"{'Lasanha', 'Abacate (Guacamole ou Salada)', '...","{Avocado (guacamole or salad), Peru, Chicken i..."
3,American Brown Ale,"{'Feijão/ Feijoada', 'Carne com Chili', 'Bife ...","{Tamboril (fish), Meat roll, Grilled lamb, Bea..."
4,German Pilsner,"{'Sopa (caldo de Piranha)', 'Caranguejo', 'Emp...","{Empanadas, Pizza Mussarela/Marguerita, Crab, ..."


In [91]:
harmonizacao_df = harmonizacao_df.replace('', np.nan)
harmonizacao_df.dropna(inplace=True)



In [101]:
# Salvar o novo DataFrame em um arquivo CSV
harmonizacao_df.to_csv("./datasets/preprocessed/HARMONIZACOES_preprocessed.csv", index=False)